In [18]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from pathlib import Path
import collections

from exp_setup import *
from coprover.results_analysis import *

In [23]:
res_fpath = Path("results", "t5_compare_pred.v1.csv")

In [2]:
v1_gg_telem = read_gg_from_csv(res_fpath)

In [5]:
print(v1_gg_telem.class_report(return_dict=False))

              precision    recall  f1-score   support

           n       0.94      0.91      0.92      5475
           p       0.91      0.94      0.92      5460

    accuracy                           0.92     10935
   macro avg       0.92      0.92      0.92     10935
weighted avg       0.92      0.92      0.92     10935



In [39]:
mean_acc, std_acc, samples = bootstrap_acc_stats(res_fpath)
print(f"Bootstrap, # samples={samples}, acc/std={mean_acc:.5f}/{std_acc:.5f}")

Bootstrap, # samples=1000, acc/std=0.92314/0.00249


In [37]:
baseline_gg_telem = read_gg_from_csv(res_fpath)
baseline_gg_telem.guesses = [POS for _ in range(len(baseline_gg_telem.guesses))]
print(baseline_gg_telem.class_report(return_dict=False))
baseline_gg_telem.save(Path("results", "baseline.v1.csv"))

/home/fennel1/yeh/anaconda/anaconda3/envs/coprovers/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/fennel1/yeh/anaconda/anaconda3/envs/coprovers/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/fennel1/yeh/anaconda/anaconda3/envs/coprovers/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

              precision    recall  f1-score   support

           n       0.00      0.00      0.00      5475
           p       0.50      1.00      0.67      5460

    accuracy                           0.50     10935
   macro avg       0.25      0.50      0.33     10935
weighted avg       0.25      0.50      0.33     10935



In [40]:
baseline_mean_acc, baseline_std_acc, baseline_samples = bootstrap_acc_stats(Path("results", "baseline.v1.csv"))
print(f"Bootstrap baseline, # samples={baseline_samples}, acc/std={baseline_mean_acc:.5f}/{baseline_std_acc:.5f}")

Bootstrap baseline, # samples=1000, acc/std=0.49936/0.00481


In [49]:
p_val = two_t(mean_acc, std_acc, len(test_df), baseline_mean_acc, baseline_std_acc, len(test_df))
print(f"PVal={p_val}")
cv1, cv2 = get_crit_vals(0.001, len(test_df))

PVal=8179.092100435865
Critical value = 3.291/-3.291, two tailed alpha=0.001, N=10935


In [6]:
train_df, test_df = setup_data()

Setting up last step pred from compare_pred.v1.csv.gz
Len train=98411, test=10935


In [32]:
collections.Counter(train_df.target_text)

Counter({'p': 49213, 'n': 49198})

In [33]:
collections.Counter(test_df.target_text)

Counter({'n': 5475, 'p': 5460})

In [22]:
# Sanity check: Check for token lengths for sequents

s1_lt_s2s = []
for _, row in tqdm(train_df.iterrows(), total=len(train_df)):
    (cmds, s1, s2) = row.source_text.split("<ANT>")
    toks_s1 = s1.strip().split()
    toks_s2 = s2.strip().split()
    if len(toks_s1) < len(toks_s2):
        s1_lt_s2s.append("LT")
    elif len(toks_s1) > len(toks_s2):
        s1_lt_s2s.append("GT")
    else:
        s1_lt_s2s.append("EQ")
print(collections.Counter(s1_lt_s2s))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 98411/98411 [00:10<00:00, 9126.06it/s]

Counter({'LT': 42764, 'GT': 42623, 'EQ': 13024})


In [28]:
# Sanity check: Check for token lengths for sequents

s1_lt_s2s = []
for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    (cmds, s1, s2) = row.source_text.split("<ANT>")
    toks_s1 = s1.strip().split()
    toks_s2 = s2.strip().split()
    if len(toks_s1) < len(toks_s2):
        s1_lt_s2s.append("LT")
    elif len(toks_s1) > len(toks_s2):
        s1_lt_s2s.append("GT")
    else:
        s1_lt_s2s.append("EQ")
print(collections.Counter(s1_lt_s2s))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10935/10935 [00:01<00:00, 8480.54it/s]

Counter({'GT': 4823, 'LT': 4682, 'EQ': 1430})
